In [ ]:
# instalação dos programas necessários
%%bash
mkdir logs
sudo apt install awk bwa samtools \
1>./logs/log_instalacao.txt \
2>./logs/erros_instalacao.txt

In [ ]:
# baixar GATK: chamador variantes (Genome Analysis Took Kit)
%%bash
wget https://github.com/broadinstitute/gatk/releases/download/4.1.8.1/gatk-4.1.8.1.zip 1>./logs/log_down_gatk.txt 2>./logs/erros_down_gatk.txt
unzip ./gatk-4.1.8.1.zip 1>./logs/log_unzip_gatk.txt 2>./logs/erros_unzip_gatk.txt
rm ./gatk-4.1.8.1.zip

In [ ]:
# baixar Picard 
%%bash
wget https://github.com/broadinstitute/picard/releases/download/2.24.2/picard.jar 1>./logs/log_down_picard.txt 2>./logs/erro_down_picard.txt

In [ ]:
# organizar os diretórios
%%bash
mkdir referencia
mkdir referencia/hg38

mkdir dados
mkdir dados/fastq
mkdir dados/bwa
mkdir dados/gatk

In [ ]:
# mover amostras e genoma de referência para seus respectivos diretórios
%%bash
mv ./amostra-lbb_R1.fq.gz ./dados/fastq
mv ./amostra-lbb_R2.fq.gz ./dados/fastq
mv ./grch38.chr22.fasta.gz ./referencia/hg38

In [ ]:
# Descompactar o arquivo gz do cromossomo 22
%%bash
gunzip ./referencia/hg38/grch38.chr22.fasta.gz

In [ ]:
# visualizar as 10 primeiras linhas
%%bash
cat ./referencia/hg38/grch38.chr22.fasta | head -n 10

>chr22
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN


In [ ]:
# indexar o genoma de referência
%%bash
bwa index -a bwtsw ./referencia/hg38/grch38.chr22.fasta

In [ ]:
# criar o índice da referência (chr22)
%%bash
samtools faidx ./referencia/hg38/grch38.chr22.fasta

In [ ]:
# CreateSequenceDictionary (algoritmo)
%%bash
java -jar picard.jar CreateSequenceDictionary \
REFERENCE=./referencia/hg38/grch38.chr22.fasta \
OUTPUT=./referencia/hg38/grch38.chr22.dict

In [ ]:
# mapear reads no genoma de geferência do chr 22
%%bash
NOME=VANESSA
Biblioteca=Exoma
Plataforma=NovaSeq6000

bwa mem -M -R "@RG\tID:CAP\tSM:$NOME\tLB:$Biblioteca\tPL:$Plataforma" \
./referencia/hg38/grch38.chr22.fasta \
./dados/fastq/amostra-lbb_R1.fq.gz \
./dados/fastq/amostra-lbb_R2.fq.gz \
1>./dados/bwa/AMOSTRA01.sam \
2>./logs/erros_bwa_mem.txt

In [ ]:
%%bash
head ./dados/bwa/AMOSTRA01.sam

@SQ	SN:chr22	LN:50818468
@RG	ID:CAP	SM:VANESSA	LB:Exoma	PL:NovaSeq6000
@PG	ID:bwa	PN:bwa	VN:0.7.17-r1188	CL:bwa mem -M -R @RG\tID:CAP\tSM:VANESSA\tLB:Exoma\tPL:NovaSeq6000 ./referencia/hg38/grch38.chr22.fasta ./dados/fastq/amostra-lbb_R1.fq.gz ./dados/fastq/amostra-lbb_R2.fq.gz
210730_A00776_0154_BHCMWGDSX2:1:1101:10041:3380	99	chr22	42140705	0	101M	=	42140793	189	GAGATGCAGGGTGAGAGTGGGGACTGGACTCTAGGATGCTGGGACCCCTGCCACCAAACACACGGGGGACACACACTGCCTGGCACACAGCTGGACTCTGT	?????????????????????????????????????????????????????????????????????????????????????????????????????	NM:i:0	MD:Z:101	MC:Z:101M	AS:i:101	XS:i:101	RG:Z:CAP	XA:Z:chr22,+42127001,101M,0;
210730_A00776_0154_BHCMWGDSX2:1:1101:10041:3380	147	chr22	42140793	0	101M	=	42140705	-189	AGCTGGACTCTGTCAACTAGTCCTGCGCCCGAGAAGCTCCACAGTACCCTCTCCGACCCCACAGCAGGGCGCAGTCACACCTCTCAGAGGCACCCACACTG	?????????????????????????????????????????????????????????????????????????????????????????????????????	NM:i:0	MD:Z:101	MC:Z:101M	AS:i:101	XS:i:101	RG:Z:CAP	

In [ ]:
%%bash
# fixmate: marca dentro do arquivo mapeado as reads que não tem o matepair mapeado 
samtools fixmate ./dados/bwa/AMOSTRA01.sam ./dados/bwa/AMOSTRA01.bam

# ordenar
samtools sort -O bam -o ./dados/bwa/AMOSTRA01_sorted.bam ./dados/bwa/AMOSTRA01.bam 

# indexar
samtools index ./dados/bwa/AMOSTRA01_sorted.bam

In [ ]:
%%bash
samtools view ./dados/bwa/AMOSTRA01_sorted.bam | head

210730_A00776_0154_BHCMWGDSX2:3:1332:17074:11177	99	chr22	10520269	60	101M	=	10520361	193	AGCCTAGCCTGCAGGTGGGGCTTCACTGGGGACCCATCACCTTCCACCCAGGAGCCTGTCTGCCTCCCACAACCATCCATGGCACCCAGGCTGCTGGCATC	?????????????????????????????????????????????????????????????????????????????????????????????????????	NM:i:0	MD:Z:101	AS:i:101	XS:i:19	RG:Z:CAP	MQ:i:60	MC:Z:101M
210730_A00776_0154_BHCMWGDSX2:3:1332:17074:11177	147	chr22	10520361	60	101M	=	10520269	-193	GCTGGCATCAAGGGGCACTTGCAGGCCAGTGCCAAGCCACCCTCGTACCCCCTCATCTTCCCCTCCCATGCTCCTGCTCCTCAGTGTCCAAAGTCCAGAAG	?????????????????????????????????????????????????????????????????????????????????????????????????????	NM:i:0	MD:Z:101	AS:i:101	XS:i:0	RG:Z:CAP	MQ:i:60	MC:Z:101M
210730_A00776_0154_BHCMWGDSX2:4:2449:23448:30890	99	chr22	10527180	60	101M	=	10527282	203	CTTTTTACCCGCGCCGCCACGGCTTTTTGTGGTTTTTTTTGCCCCCACTCCCGCTGCTTTTTGCTCCTGCCACCGCGGCTTTTTGCCCCCGCCGCGGATTT	???????????????????????55??????????????????????????????????????????????'???????5???????????????????

In [ ]:
# HaplotypeCaller: algoritmo para identificar variantes
%%bash
gatk-4.1.8.1/gatk HaplotypeCaller \
-R ./referencia/hg38/grch38.chr22.fasta \
-I ./dados/bwa/AMOSTRA01_sorted.bam \
-O ./dados/gatk/AMOSTRA01_sorted.vcf \
-bamout ./dados/bwa/AMOSTRA01_sorted_bamout.bam

In [ ]:
%%bash
cat ./dados/gatk/AMOSTRA01_sorted.vcf | grep -v "^##" | head

#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	VANESSA
chr22	10553503	.	T	G	37.32	.	AC=2;AF=1.00;AN=2;DP=2;ExcessHet=3.0103;FS=0.000;MLEAC=1;MLEAF=0.500;MQ=60.00;QD=18.66;SOR=0.693	GT:AD:DP:GQ:PL	1/1:0,2:2:6:49,6,0
chr22	10637059	.	C	T	2551.06	.	AC=2;AF=1.00;AN=2;BaseQRankSum=-0.430;DP=83;ExcessHet=3.0103;FS=13.491;MLEAC=2;MLEAF=1.00;MQ=58.46;MQRankSum=4.858;QD=30.74;ReadPosRankSum=2.747;SOR=0.472	GT:AD:DP:GQ:PL	1/1:4,79:83:99:2565,111,0
chr22	10637150	.	T	G	1199.06	.	AC=2;AF=1.00;AN=2;DP=43;ExcessHet=3.0103;FS=0.000;MLEAC=2;MLEAF=1.00;MQ=53.19;QD=34.26;SOR=7.168	GT:AD:DP:GQ:PL	1/1:0,35:35:99:1213,105,0
chr22	10685457	.	A	C	58.32	.	AC=2;AF=1.00;AN=2;DP=2;ExcessHet=3.0103;FS=0.000;MLEAC=1;MLEAF=0.500;MQ=60.00;QD=29.16;SOR=2.303	GT:AD:DP:GQ:PL	1/1:0,2:2:6:70,6,0
chr22	10685500	.	G	A	58.32	.	AC=2;AF=1.00;AN=2;DP=2;ExcessHet=3.0103;FS=0.000;MLEAC=1;MLEAF=0.500;MQ=60.00;QD=29.16;SOR=2.303	GT:AD:DP:GQ:PL	1/1:0,2:2:6:70,6,0
chr22	10685693	.	A	G	38.64	.	AC=1;AF=0.500;AN=2;BaseQRankSum=2.100;DP=

In [ ]:
# quantas variantes foram encontradas
%%bash
cat ./dados/gatk/AMOSTRA01_sorted.vcf | grep -v "^#" | wc -l

3609


In [ ]:
# variantes do GABARITO
%%bash
cat ./pequeno-gabarito.vcf | grep -v "^##" 

#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	AMOSTRA-LBB
chr22	19039100	.	A	G	.	PASS	.	GT	0/1
chr22	19764306	.	C	T	.	PASS	.	GT	0/1
chr22	21975710	.	T	C	.	PASS	.	GT	0/1
chr22	23069838	.	A	G	.	PASS	.	GT	0/1
chr22	26027014	.	A	G	.	PASS	.	GT	1/1
chr22	29441577	.	TTCC	T	.	PASS	.	GT	0/1
chr22	33006594	.	C	T	.	PASS	.	GT	0/1
chr22	35264882	.	G	T	.	PASS	.	GT	1/1
chr22	37746461	.	A	G	.	PASS	.	GT	1/1
chr22	43180898	.	G	A	.	PASS	.	GT	0/1


In [ ]:
# checar se algumas variantes batem com o gabarito
%%bash
grep '19039100' ./dados/gatk/AMOSTRA01_sorted.vcf | awk '{print $2,$4,$5}'
grep '29441577' ./dados/gatk/AMOSTRA01_sorted.vcf | awk '{print $2,$4,$5}'
grep '43180898' ./dados/gatk/AMOSTRA01_sorted.vcf | awk '{print $2,$4,$5}' 

19039100 A G
29441577 TTCC T
43180898 G A
